In [ ]:
# Done with python 3.9.6, osx
!pip install PyPDF2

In [2]:
import os
import PyPDF2

In [5]:
path = "/Users/dee/Downloads/Books/"
things = os.listdir(path)
things = [os.path.join(path, p) for p in things]
books = []
for thing in things:
    if ".epub" in thing or ".pdf" in thing:
        books.append(thing)

In [6]:
books

['/Users/dee/Downloads/Books/Jazz Ballads - Jazz Piano Solos Series Volume 10.epub',
 '/Users/dee/Downloads/Books/Cocktail Jazz - Jazz Piano Solos Series Volume 46.epub',
 '/Users/dee/Downloads/Books/Bill Evans Fake Book.pdf',
 '/Users/dee/Downloads/Books/Gypsy.pdf',
 '/Users/dee/Downloads/Books/Gypsy jazz - Jazz Piano Solos Series Volume 20.pdf',
 '/Users/dee/Downloads/Books/Pop Standards - Jazz Piano Solos Series Volume 41.pdf']

In [19]:
def resize_pdf(iPath, oPath):
    with open(iPath, 'rb') as file:
        
        txt = file.readlines()
        al = 0
        for i, x in enumerate(txt[::-1]):
            if b'%%EOF' in x:
                al = len(txt)-i-1
                break
        txtx = txt[:al] + [b'%%EOF']

        with open(iPath, 'wb') as f:
            f.writelines(txtx)

        pdf_reader = PyPDF2.PdfReader(iPath)
        pdf_writer = PyPDF2.PdfWriter()

        for page_num in range(pdf_reader.numPages):
            page = pdf_reader.getPage(page_num)
            print(page.mediaBox.lower_left)
            page.mediaBox.lower_left=(0, 0)
            page.mediaBox.upper_right=(page.mediaBox.getWidth(), page.mediaBox.getHeight())
            pdf_writer.addPage(page)
        
        with open(oPath, 'wb') as oFile:
            pdf_writer.write(oFile)

In [20]:
out_dir = "/Users/dee/Downloads/Books/processed"
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
for book in books:
    if ".epub" in book:
        out = os.path.join(out_dir, os.path.basename(book))
        resize_pdf(book, out)

PdfReadError: EOF marker not found